<a href="https://colab.research.google.com/github/Ashishat404/Data-Science/blob/main/LocalCodeSearchEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Install required libraries
!pip install rich colorama --quiet

In [19]:
import os
import re
import ast
import math
from pathlib import Path
from collections import defaultdict
from typing import List, Dict, Tuple

# For pretty output in Colab
from IPython.display import display, HTML

print("✅ All imports successful!")

✅ All imports successful!


In [20]:
from google.colab import files
import io

# ── Upload files ──────────────────────────────────────────────
print("📂 Upload your code files (Python, JS, Java, C++, etc.)")
uploaded = files.upload()

# Store file contents in a dict {filename: content}
CODE_FILES: Dict[str, str] = {}

for filename, content_bytes in uploaded.items():
    try:
        content = content_bytes.decode("utf-8")
    except UnicodeDecodeError:
        content = content_bytes.decode("latin-1")
    CODE_FILES[filename] = content
    print(f"  ✅ Loaded: {filename} ({len(content.splitlines())} lines)")

print(f"\n📦 Total files loaded: {len(CODE_FILES)}")

📂 Upload your code files (Python, JS, Java, C++, etc.)


Saving AgeCalculator - Copy.java to AgeCalculator - Copy (1).java
Saving BankingApplication - Copy.java to BankingApplication - Copy (1).java
Saving EventHandling - Copy.java to EventHandling - Copy (1).java
Saving J_09_OperatorPrecedence - Copy.java to J_09_OperatorPrecedence - Copy (1).java
  ✅ Loaded: AgeCalculator - Copy (1).java (17 lines)
  ✅ Loaded: BankingApplication - Copy (1).java (75 lines)
  ✅ Loaded: EventHandling - Copy (1).java (22 lines)
  ✅ Loaded: J_09_OperatorPrecedence - Copy (1).java (24 lines)

📦 Total files loaded: 4


In [21]:
import string

def tokenize(text: str) -> List[str]:
    """Convert code text into lowercase tokens (words/identifiers)."""
    # Split on non-word characters, keep meaningful tokens
    tokens = re.findall(r'\b[a-zA-Z_][a-zA-Z0-9_]*\b', text)
    return [t.lower() for t in tokens]

def build_inverted_index(code_files: Dict[str, str]) -> Dict[str, List[str]]:
    """
    Build an inverted index: {token -> [list of filenames containing it]}
    """
    index = defaultdict(set)
    for filename, content in code_files.items():
        tokens = set(tokenize(content))
        for token in tokens:
            index[token].add(filename)
    return {k: list(v) for k, v in index.items()}

# Build index
INVERTED_INDEX = build_inverted_index(CODE_FILES)
print(f"📇 Index built with {len(INVERTED_INDEX)} unique tokens.")

📇 Index built with 132 unique tokens.


In [22]:
def keyword_search(query: str, code_files: Dict[str, str]) -> List[Dict]:
    """
    Search for a keyword/phrase across all code files.
    Returns list of results with filename, line number, and matched line.
    """
    query_lower = query.lower()
    results = []

    for filename, content in code_files.items():
        lines = content.splitlines()
        matches = []
        for line_no, line in enumerate(lines, start=1):
            if query_lower in line.lower():
                matches.append({
                    "line_no": line_no,
                    "line": line.strip(),
                })
        if matches:
            results.append({
                "filename": filename,
                "match_count": len(matches),
                "matches": matches,
            })

    # Sort by number of matches (descending)
    results.sort(key=lambda x: x["match_count"], reverse=True)
    return results


def display_keyword_results(query: str, results: List[Dict]):
    """Pretty-print keyword search results."""
    print(f"\n🔍 Keyword Search Results for: '{query}'")
    print("=" * 60)
    if not results:
        print("  ❌ No matches found.")
        return
    for r in results:
        print(f"\n📄 File: {r['filename']}  ({r['match_count']} match(es))")
        print("-" * 50)
        for m in r["matches"][:5]:  # Show first 5 matches per file
            print(f"  Line {m['line_no']:>4}: {m['line']}")
        if len(r["matches"]) > 5:
            print(f"  ... and {len(r['matches']) - 5} more match(es)")

In [23]:
def detect_functions_java(content: str) -> List[Dict]:
    """
    Enhanced Java-specific function/class/method detector.
    """
    patterns = [
        # Class declaration
        (r'^\s*(?:public|private|protected)?\s*(?:abstract|final)?\s*class\s+(\w+)', "class"),
        # Interface declaration
        (r'^\s*(?:public|private|protected)?\s*interface\s+(\w+)', "interface"),
        # Constructor: ClassName(...)
        (r'^\s*(?:public|private|protected)\s+(\w+)\s*\(', "constructor"),
        # Method: returnType methodName(...)
        (r'^\s*(?:public|private|protected|static|final|synchronized|void|\w+)\s+(?:static\s+)?(?:final\s+)?(\w+)\s*\([^;]*$', "method"),
    ]

    functions = []
    seen = set()  # avoid duplicates
    for line_no, line in enumerate(content.splitlines(), start=1):
        for pattern, ftype in patterns:
            m = re.match(pattern, line.strip(), re.IGNORECASE)
            if m:
                name = m.group(1)
                key = (name, line_no)
                if key not in seen:
                    seen.add(key)
                    functions.append({
                        "type": ftype,
                        "name": name,
                        "line": line_no,
                        "args": [],
                    })
                break  # only match one pattern per line
    return functions


# ── Patch detect_all_functions to use Java detector ──────────
def detect_all_functions(code_files: Dict[str, str]) -> Dict[str, List[Dict]]:
    """Detect functions/classes in all uploaded files (Java-aware)."""
    all_detections = {}
    for filename, content in code_files.items():
        if filename.endswith(".py"):
            funcs = detect_functions_python(content, filename)
            if not funcs:
                funcs = detect_functions_generic(content)
        elif filename.endswith(".java"):
            funcs = detect_functions_java(content)  # ✅ Java-specific
        else:
            funcs = detect_functions_generic(content)
        all_detections[filename] = funcs
    return all_detections

print("✅ Java-enhanced detector loaded! Now re-run the search engine cell.")

✅ Java-enhanced detector loaded! Now re-run the search engine cell.


In [24]:
def compute_tf(content: str) -> Dict[str, float]:
    """Term Frequency: how often each token appears in a file."""
    tokens = tokenize(content)
    if not tokens:
        return {}
    freq = defaultdict(int)
    for t in tokens:
        freq[t] += 1
    total = len(tokens)
    return {t: count / total for t, count in freq.items()}


def compute_idf(code_files: Dict[str, str]) -> Dict[str, float]:
    """Inverse Document Frequency: penalizes tokens common across all files."""
    N = len(code_files)
    doc_freq = defaultdict(int)
    for content in code_files.values():
        unique_tokens = set(tokenize(content))
        for t in unique_tokens:
            doc_freq[t] += 1
    return {t: math.log((N + 1) / (df + 1)) + 1 for t, df in doc_freq.items()}


def rank_files(query: str, code_files: Dict[str, str]) -> List[Tuple[str, float]]:
    """
    Rank files by TF-IDF score for a given query.
    Returns sorted list of (filename, score).
    """
    query_tokens = tokenize(query)
    idf = compute_idf(code_files)

    scores = {}
    for filename, content in code_files.items():
        tf = compute_tf(content)
        score = sum(tf.get(t, 0) * idf.get(t, 0) for t in query_tokens)
        scores[filename] = score

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked


def display_ranked_files(query: str, ranked: List[Tuple[str, float]]):
    """Display file ranking results."""
    print(f"\n📊 File Ranking for query: '{query}'")
    print("=" * 60)
    if not ranked:
        print("  ❌ No files to rank.")
        return
    max_score = ranked[0][1] if ranked[0][1] > 0 else 1
    for i, (filename, score) in enumerate(ranked, start=1):
        bar_len = int((score / max_score) * 30) if max_score > 0 else 0
        bar = "█" * bar_len + "░" * (30 - bar_len)
        print(f"  {i:>2}. [{bar}] {score:.4f}  📄 {filename}")

In [25]:
def run_search_engine():
    """Interactive CLI-style search engine for Google Colab."""
    if not CODE_FILES:
        print("⚠️  No files loaded! Please run Step 3 first.")
        return

    # Pre-compute function detections
    ALL_FUNCTIONS = detect_all_functions(CODE_FILES)

    print("\n" + "=" * 60)
    print("  🔍  LOCAL CODE SEARCH ENGINE  ")
    print("=" * 60)
    print("Commands:")
    print("  search <keyword>   — Keyword search across files")
    print("  rank <query>       — Rank files by relevance")
    print("  functions          — List all detected functions/classes")
    print("  files              — List all loaded files")
    print("  quit               — Exit")
    print("=" * 60)

    while True:
        try:
            user_input = input("\n> ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\n👋 Exiting search engine.")
            break

        if not user_input:
            continue

        parts = user_input.split(maxsplit=1)
        cmd = parts[0].lower()
        arg = parts[1] if len(parts) > 1 else ""

        if cmd == "quit" or cmd == "exit":
            print("👋 Goodbye!")
            break

        elif cmd == "search":
            if not arg:
                print("  ⚠️  Usage: search <keyword>")
            else:
                results = keyword_search(arg, CODE_FILES)
                display_keyword_results(arg, results)

        elif cmd == "rank":
            if not arg:
                print("  ⚠️  Usage: rank <query>")
            else:
                ranked = rank_files(arg, CODE_FILES)
                display_ranked_files(arg, ranked)

        elif cmd == "functions":
            display_functions(ALL_FUNCTIONS)

        elif cmd == "files":
            print(f"\n📦 Loaded Files ({len(CODE_FILES)}):")
            for i, (fname, content) in enumerate(CODE_FILES.items(), 1):
                lines = len(content.splitlines())
                print(f"  {i:>2}. {fname}  ({lines} lines)")

        else:
            print(f"  ❓ Unknown command: '{cmd}'. Type 'search', 'rank', 'functions', 'files', or 'quit'.")


# ── Launch the engine ──────────────────────────────────────────
run_search_engine()


  🔍  LOCAL CODE SEARCH ENGINE  
Commands:
  search <keyword>   — Keyword search across files
  rank <query>       — Rank files by relevance
  functions          — List all detected functions/classes
  files              — List all loaded files
  quit               — Exit

> functions

🔎 Detected Functions & Classes

📄 AgeCalculator - Copy (1).java — 1 definition(s) found
--------------------------------------------------
  🏛️ [          CLASS]  Line    4:  AgeCalculator()

📄 BankingApplication - Copy (1).java — 9 definition(s) found
--------------------------------------------------
  🏛️ [          CLASS]  Line    3:  InsufficientBalanceException()
  ⚙️ [    CONSTRUCTOR]  Line    4:  InsufficientBalanceException()
  🏛️ [          CLASS]  Line    9:  ExceedTransactionLimitException()
  ⚙️ [    CONSTRUCTOR]  Line   10:  ExceedTransactionLimitException()
  🏛️ [          CLASS]  Line   16:  ExcessiveTransactionAmountException()
  ⚙️ [    CONSTRUCTOR]  Line   17:  ExcessiveTransactionAmoun